In [1]:
# Importing all the necessary libaries that will be used through the project

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import geopandas

In [2]:
state = geopandas.read_file(
    "../ne_110m_admin_1_states_provinces/ne_110m_admin_1_states_provinces.shp"
)
state.head()

,featurecla,scalerank,adm1_code,diss_me,iso_3166_2,wikipedia,iso_a2,adm0_sr,name,name_alt,...,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,FCLASS_TLC,geometry
0,Admin-1 scale rank,2,USA-3514,3514,US-MN,http://en.wikipedia.org/wiki/Minnesota,US,1,Minnesota,MN|Minn.,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-89.95766 47.28691, -90.13175 47.292..."
1,Admin-1 scale rank,2,USA-3515,3515,US-MT,http://en.wikipedia.org/wiki/Montana,US,1,Montana,MT|Mont.,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-116.04823 49.00037, -113.05950 49.0..."
2,Admin-1 scale rank,2,USA-3516,3516,US-ND,http://en.wikipedia.org/wiki/North_Dakota,US,1,North Dakota,ND|N.D.,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-97.22894 49.00089, -97.21414 48.902..."
3,Admin-1 scale rank,2,USA-3517,3517,US-HI,http://en.wikipedia.org/wiki/Hawaii,US,8,Hawaii,HI|Hawaii,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-155.93665 19.05939, -155.9080..."
4,Admin-1 scale rank,2,USA-3518,3518,US-ID,http://en.wikipedia.org/wiki/Idaho,US,1,Idaho,ID|Idaho,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-116.04823 49.00037, -115.96780 47.9..."


In [3]:
# Read in the 1980 and 2021 data to compare air quality in lockdown and before lockdown

oldAir_df = pd.read_csv("../daily_aqi_by_county_1980.csv")
newAir_df = pd.read_csv("../daily_aqi_by_county_2024.csv")

In [4]:
# Group by "State Name" and calculate the mean AQI for oldAir_df
oldAir_mean_aqi = oldAir_df.groupby("State Name")["AQI"].mean().reset_index()

# Group by "State Name" and calculate the mean AQI for newAir_df
newAir_mean_aqi = newAir_df.groupby("State Name")["AQI"].mean().reset_index()


# Display the results
print(oldAir_mean_aqi)
print(newAir_mean_aqi)

              State Name        AQI
0                Alabama  65.240076
1                 Alaska  45.604414
2                Arizona  93.704272
3               Arkansas  41.818182
4             California  68.606929
5               Colorado  49.718805
6            Connecticut  89.888818
7               Delaware  53.458580
8   District Of Columbia  79.226776
9                Florida  45.447225
10               Georgia  35.517600
11                Hawaii  16.803324
12                 Idaho  90.588599
13              Illinois  58.990562
14               Indiana  66.820131
15                  Iowa  42.738400
16                Kansas  49.538889
17              Kentucky  66.849469
18             Louisiana  39.367008
19                 Maine  65.496812
20              Maryland  64.122608
21         Massachusetts  62.852991
22              Michigan  46.613220
23             Minnesota  47.684480
24           Mississippi  35.997091
25              Missouri  72.186503
26               Montana  51

In [5]:
# Merge the state geodataset with the old AQI dataset
full_old = pd.merge(
    state, oldAir_mean_aqi, left_on="name", right_on="State Name", how="outer"
)

# Merge the state geodataset with the new AQI dataset
full_new = pd.merge(
    state, newAir_mean_aqi, left_on="name", right_on="State Name", how="outer"
)

In [6]:
full_old.head(5)
list(full_old.columns)


['featurecla',
 'scalerank',
 'adm1_code',
 'diss_me',
 'iso_3166_2',
 'wikipedia',
 'iso_a2',
 'adm0_sr',
 'name',
 'name_alt',
 'name_local',
 'type',
 'type_en',
 'code_local',
 'code_hasc',
 'note',
 'hasc_maybe',
 'region',
 'region_cod',
 'provnum_ne',
 'gadm_level',
 'check_me',
 'datarank',
 'abbrev',
 'postal',
 'area_sqkm',
 'sameascity',
 'labelrank',
 'name_len',
 'mapcolor9',
 'mapcolor13',
 'fips',
 'fips_alt',
 'woe_id',
 'woe_label',
 'woe_name',
 'latitude',
 'longitude',
 'sov_a3',
 'adm0_a3',
 'adm0_label',
 'admin',
 'geonunit',
 'gu_a3',
 'gn_id',
 'gn_name',
 'gns_id',
 'gns_name',
 'gn_level',
 'gn_region',
 'gn_a1_code',
 'region_sub',
 'sub_code',
 'gns_level',
 'gns_lang',
 'gns_adm1',
 'gns_region',
 'min_label',
 'max_label',
 'min_zoom',
 'wikidataid',
 'name_ar',
 'name_bn',
 'name_de',
 'name_en',
 'name_es',
 'name_fr',
 'name_el',
 'name_hi',
 'name_hu',
 'name_id',
 'name_it',
 'name_ja',
 'name_ko',
 'name_nl',
 'name_pl',
 'name_pt',
 'name_ru',
 'na

In [7]:
# Rename columns for convenience
full_old = full_old.rename(columns={"State Name": "State", "AQI": "AQI"})
full_new = full_new.rename(columns={"State Name": "State", "AQI": "AQI"})

# Verify data structure
print(full_old[["State", "AQI"]].head())

          State        AQI
0     Minnesota  47.684480
1       Montana  51.760739
2  North Dakota  24.546099
3        Hawaii  16.803324
4         Idaho  90.588599


In [10]:
import folium
from folium import Choropleth
import geopandas as gpd

# Filter out rows with missing geometries
full_old = full_old[full_old.geometry.notnull()]

# Create GeoDataFrame
full_old_gdf = gpd.GeoDataFrame(full_old, geometry=full_old.geometry)
if full_old_gdf.crs != "EPSG:4326":
    full_old_gdf = full_old_gdf.to_crs("EPSG:4326")

# Initialize the map with consistent dimensions and zoom
m = folium.Map(location=[37.8, -96.9], zoom_start=4)

# Add Choropleth for AQI data
Choropleth(
    geo_data=full_old_gdf,
    data=full_old_gdf,
    columns=["State", "AQI"],
    key_on="feature.properties.name",
    fill_color="Reds",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Average AQI (1980)",
    threshold_scale=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
).add_to(m)

# Add GeoJson layer for South Dakota and Wyoming with a pattern overlay
pattern_states = ["South Dakota", "Wyoming"]
for state in pattern_states:
    state_geom = full_old_gdf[full_old_gdf['name'] == state]
    if state_geom.empty:
        continue  # Skip if no geometry is available for the state
    folium.GeoJson(
        state_geom,
        style_function=lambda x: {
            'fillColor': 'url(#diagonal-stripe)',  # Use SVG pattern
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7,
        },
        tooltip=folium.GeoJsonTooltip(
            fields=["name", "AQI"], aliases=["State:", "Average AQI:"], localize=True
        ),
    ).add_to(m)

# Inject SVG pattern into the map
pattern_svg = """
<svg xmlns="http://www.w3.org/2000/svg">
  <defs>
    <pattern id="diagonal-stripe" patternUnits="userSpaceOnUse" width="10" height="10">
      <line x1="0" y1="0" x2="10" y2="10" stroke="black" stroke-width="1" />
      <line x1="10" y1="0" x2="0" y2="10" stroke="black" stroke-width="1" />
    </pattern>
  </defs>
</svg>
"""
m.get_root().html.add_child(folium.Element(pattern_svg))

# Add a GeoJson layer for transparent states and border consistency
folium.GeoJson(
    full_old_gdf,
    style_function=lambda x: {
        "fillColor": "transparent",
        "color": "black",
        "weight": 0.5,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["name", "AQI"], aliases=["State:", "Average AQI:"], localize=True
    ),
).add_to(m)

# Save the map
m.save("aqi_choropleth_1980.html")
m


In [11]:
import folium
from folium import Choropleth

# Filter out rows with missing geometries
full_new = full_new[full_new.geometry.notnull()]

full_new_gdf = geopandas.GeoDataFrame(full_new, geometry=full_new.geometry)
if full_new_gdf.crs != "EPSG:4326":
    full_new_gdf = full_new_gdf.to_crs("EPSG:4326")

m = folium.Map(location=[37.8, -96.9], zoom_start=4)
Choropleth(
    geo_data=full_new_gdf,
    data=full_new_gdf,
    columns=["State", "AQI"],
    key_on="feature.properties.name",
    fill_color="Reds",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Average AQI (2024)",
    threshold_scale=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
).add_to(m)

folium.GeoJson(
    full_new_gdf,
    style_function=lambda x: {
        "fillColor": "transparent",
        "color": "black",
        "weight": 0.5,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["State", "AQI"], aliases=["State:", "Average AQI:"], localize=True
    ),
).add_to(m)

m.save("aqi_choropleth_2024.html")
m